1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
import pandas as pd
import numpy as np
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
print('Max DataSize:',data.shape[0])
N = np.random.randint(10000, data.shape[0])
df = data[:N]
df

Max DataSize: 1310720


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
726466,1,1,3,3869207163,2226,15
726467,1,1,4,3869207163,2230,14
726468,1,1,6,3869207163,2230,20
726469,1,0,47,3869207163,2351,11


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [4]:
x = df['BX_COUNTER'].max()
x

3563

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [7]:
tdc_count_ns = 25/30
bx_unit_ns = 25 
dfa = df.copy()

orbit_time = (dfa['ORBIT_CNT'] * bx_unit_ns * x) - (dfa.loc[0,'ORBIT_CNT'] * bx_unit_ns * x)
bx_time = (dfa['BX_COUNTER'] * bx_unit_ns) - (dfa['BX_COUNTER'] * bx_unit_ns)[0]
tdc = (dfa['TDC_MEAS'] * tdc_count_ns) - (dfa['TDC_MEAS'] * tdc_count_ns)[0]

dfa['TIME'] = (orbit_time.cumsum() + bx_time.cumsum() + tdc.cumsum())
dfa['TIME_STAMP'] = pd.to_timedelta(dfa['TIME'])
dfa

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,TIME_STAMP
0,1,0,123,3869200167,2374,26,0.000000e+00,0 days 00:00:00
1,1,0,124,3869200167,2374,27,8.333333e-01,0 days 00:00:00
2,1,0,63,3869200167,2553,28,4.477500e+03,0 days 00:00:00.000004477
3,1,0,64,3869200167,2558,19,9.071667e+03,0 days 00:00:00.000009071
4,1,0,64,3869200167,2760,25,1.872083e+04,0 days 00:00:00.000018720
...,...,...,...,...,...,...,...,...
726466,1,1,3,3869207163,2226,15,2.350610e+14,2 days 17:17:40.988940991
726467,1,1,4,3869207163,2230,14,2.350616e+14,2 days 17:17:41.612106081
726468,1,1,6,3869207163,2230,20,2.350622e+14,2 days 17:17:42.235271176
726469,1,0,47,3869207163,2351,11,2.350629e+14,2 days 17:17:42.858439289


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [10]:
noisy_channels = df.groupby('TDC_CHANNEL')['ORBIT_CNT'].count().sort_values(ascending = False)
noisy_channels[0:3]

TDC_CHANNEL
139    9053
64     5468
63     5343
Name: ORBIT_CNT, dtype: int64

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [11]:
len(df.groupby('ORBIT_CNT').count())

1082

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [12]:
df[df['TDC_CHANNEL'] == 139].groupby('ORBIT_CNT').count().shape[0]

1081

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.